<a href="https://colab.research.google.com/github/ijnuegnaj/RAG_PRACTICE/blob/practice6/RAG%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RAG 구현
검색 -> 증강 -> 생성(LLM을 빌려서 수행)

In [ ]:
#langchain과 관련된 베이스 라이브러리들.
!pip install langchain openai langchain-community


#tiktoken은 언어의 '토큰화'를 지원해줌
!pip install tiktoken


#bm25는 벡터의 유사도 검색을 잘 해내는 라이브러리임
!pip install rank_bm25


#트랜스포머를 기반으로 문장이나 문서를 임베딩할 수 있도록 돕는 라이브러리
!pip install sentence-transformers


#벡터 데이터를 저장함(벡터 DB)
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
import tiktoken, openai

#'임베딩'을 담당하는 라이브러리.
#허깅페이스의 특정 모델을 기반으로 임베딩을 도와주는 것이 HuggingFace~~~Embeddings 임
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings

#문서를 가져온 후, 문서를 일정한 길이로 잘라줌(Text->Splitter)
from langchain.text_splitter import RecursiveCharacterTextSplitter

#VectorDB. 데이터를 임베딩한 후 숫자로 변한 데이터를 저장하고,
#DB내에서 숫자 데이터의 검색을 더 빠르게 할 수 있도록 지원함
from langchain.vectorstores import Chroma
#from langchain_community.vectorstores import FAISS > cpu용 FAISS, gpu용 FAISS

#pdf 파일이나 문서 파일을 입력했을 때, 내부의 데이터를 가져옴
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

#웹사이트 텍스트를 가져오는 역할
from langchain.document_loaders import WebBaseLoader

In [ ]:
#토크나이저는 -> 토큰화를 시키는 것
#'cl100k_base' -> 토크나이저의 종류, gpt가 사용하는 토크나이저와 유사한 특징을 가짐
tokenizer = tiktoken.get_encoding('cl100k_base')


#인풋된 text가 정해진 토크나이저로 잘려 나가면, 그 길이를 알 수 있게 됨
def titoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)


#모델 다운로드
model_hug = HuggingFaceBgeEmbeddings(model_name='BAAI/bge-m3')

/tmp/ipython-input-1805563933.py:13: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model_hug = HuggingFaceBgeEmbeddings(model_name='BAAI/bge-m3')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
loaders=[
    WebBaseLoader("https://github.com/huggingface"),
    WebBaseLoader("https://wikidocs.net/229987"),
    WebBaseLoader("https://en.wikipedia.org/wiki/Hugging_Face"),
]

In [ ]:
#데이터를 토큰화 하여 load 하기 위한 내용
docs = []

for load in loaders:
  # ~~Loader로 가져온 데이터에 대해 데이터를 불러오고 가져옴 : load_and_split()
  docs.extend(load.load_and_split())

print(f'총 문서의 길이 : {len(docs)}')

총 문서의 길이 : 25


In [ ]:
docs[0]

Document(metadata={'source': 'https://github.com/huggingface', 'title': 'Hugging Face Â· GitHub', 'description': 'The AI community building the future. Hugging Face has 339 repositories available. Follow their code on GitHub.', 'language': 'en'}, page_content='Hugging Face Â· GitHub\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu\n\nToggle navigation\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n            Sign in\n          \n\n\n \n\n\nAppearance settings\n\n\n\n\n\n\nhuggingface\n\n\n\n\n\n\n\n        Product\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            GitHub Copilot\n          \n        Write better code with AI\n      \n\n\n\n\n\n\n\n\n            GitHub Spark\n              \n                New\n              \n\n        Build and deploy intelligent apps\n      \n\n\n\n\n\n\n\n\n            GitHub Models\n              \n                New\n              \n\n        Manage and 

In [ ]:
docs[15].page_content

'GitPython: Git 저장소의 작업 조작 라이브러리\n                        \n\n\n\n\n\n                        \n                        PRAW: Python Reddit API Wrapper\n                        \n\n\n\n\n\n                        \n                        Tweepy: 파이썬 Twitter API\n                        \n\n\n\n\n\n                        \n                        facebook-sdk: 페이스북 그래프 API\n                        \n\n\n\n\n\n                        \n                        google-api-python-client: 구글 서비스 API 라이브러리\n                        \n\n\n\n\n\n                        \n                        python-slack-sdk: Slack 개발자 Kit\n                        \n\n\n\n\n\n                        \n                        python-telegram-bot: Telegram Bot API 개발 도구\n                        \n\n\n\n\n\n                        \n                        twython: Twitter API 라이브러리\n                        \n\n\n\n\n\n                        \n                        클라우드 서비스 및 API\n                        \n

In [ ]:
#토큰화(청크화)
#chunk_size = 문서(Document)를 얼마 단위로 자를까?
#chunk_overlap
#나는 코딩하는 것을 좋아합니다.
#나는 코딩하는 것을
#         하는 것을 좋아합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 250,
                                               chunk_overlap = 20,
                                               length_function = titoken_len)

texts = text_splitter.split_documents(docs)
print(f'자른 내용 : {texts[0]}, 총 길이 {len(texts)}')

자른 내용 : page_content='Hugging Face Â· GitHub













































Skip to content













Navigation Menu

Toggle navigation




 













            Sign in
          


 


Appearance settings






huggingface







        Product
        














            GitHub Copilot
          
        Write better code with AI
      








            GitHub Spark
              
                New
              

        Build and deploy intelligent apps
      








            GitHub Models
              
                New
              

        Manage and compare prompts
      








            GitHub Advanced Security
          
        Find and fix vulnerabilities
      








            Actions
          
        Automate any workflow
      














            Codespaces
          
        Instant dev environments
      








            Issues
          
        Plan and track work
      








            Code Revie

In [ ]:
from langchain.retrievers import BM25Retriever

#BM25 TF-IDF(문서 빈도-역빈도)계열의 준수한 성능을 보이는 알고리즘
bm25_r = BM25Retriever.from_documents(texts)

#질문과 가장 닮은 내용을 담고 있는 top 3개를 뽑아라~
bm25_r.k = 3

In [ ]:
result = bm25_r.invoke('What is the huggingface?')
result[0].page_content

'ArticleTalk\n\n\n\n\n\nEnglish\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nReadEditView history\n\n\n\n\n\n\n\nTools\n\n\n\n\n\nTools\nmove to sidebar\nhide\n\n\n\n\t\tActions\n\t\n\n\nReadEditView history\n\n\n\n\n\n\t\tGeneral\n\t\n\n\nWhat links hereRelated changesUpload filePermanent linkPage informationCite this pageGet shortened URLDownload QR code\n\n\n\n\n\n\t\tPrint/export\n\t\n\n\nDownload as PDFPrintable version\n\n\n\n\n\n\t\tIn other projects\n\t\n\n\nWikimedia CommonsWikidata item\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\nmove to sidebar\nhide\n\n\n\n\n\n\n\n\n\n\nFrom Wikipedia, the free encyclopedia'

In [ ]:
result = bm25_r.invoke('What is the huggingface?')
result[1].page_content

'Services and technologies[edit]\nTransformers Library[edit]\nThe Transformers library is a Python package that contains open-source implementations of transformer models for text, image, and audio tasks. It is mainly compatible with the PyTorch library, but previous versions were also compatible with TensorFlow and JAX deep learning libraries. It includes implementations of notable models like BERT and GPT-2.[20] The library was originally called "pytorch-pretrained-bert"[21] which was then renamed to "pytorch-transformers" and finally "transformers."\nA JavaScript version (Transformers.js[22]) has also been developed, allowing models to run directly in the browser through the ONNX runtime.\n\nHugging Face Hub[edit]\nThe Hugging Face Hub is a platform (centralized web service) for hosting:[23]'

In [ ]:
result = bm25_r.invoke('What is the huggingface?')
result[2].page_content

'Category\n\n\n\n\n\nRetrieved from "https://en.wikipedia.org/w/index.php?title=Hugging_Face&oldid=1304314872"\nCategories: Machine learningOpen-source artificial intelligencePrivately held companies based in New York CityAmerican companies established in 20162016 establishments in New York CityHidden categories: Articles with short descriptionShort description is different from WikidataArticles lacking reliable references from February 2024All articles lacking reliable referencesArticles with a promotional tone from August 2025All articles with a promotional tone\n\n\n\n\n\n\n This page was last edited on 5 August 2025, at 08:19\xa0(UTC).\nText is available under the Creative Commons Attribution-ShareAlike 4.0 License;\nadditional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia¬Æ is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization.\n\n\nPrivacy policy\nAbout Wikipedia\nDisclaimers\nContact Wikipedia\nC

In [ ]:
import os
from google.colab import userdata

os.environ['OPEN_API_KEY'] = userdata.get('ssu')

#리트리버 관련 모듈
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

In [ ]:
#채팅할 대상인 llm 모델 객체 만듦
llm = ChatOpenAI()

#우리가 잘라 둔 text를 VectorDB에 넣어줌
#왜? 벡터(숫자)로 만들고 빠르게 '계산' 할 수 있게 하기 위해서 => '유사도' 검색
#Chroma dB에 texts를 model_hug 방법으로 임베딩해서, 넣어놓음
chroma_db = Chroma.from_documents(texts, model_hug)

ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_kwargs': {}, 'nam...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
#리트리버를 정의
multi_query = MultiQueryRetriever.from_llm(retriever=chroma_db.as_retriever(),
                                           llm=llm)

response = multi_query.invoke('What is the huggingface?')
print(response)


In [ ]:
from langchains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, #이야기를 나눔 llm
                                 chain_type = 'studff', #
                                retriever=multi_r)

response = qa('허깅페이스가 뭐야?')
print(response['result'])